# 位置编码 (Positional Encoding)

本 Notebook 讲解 Transformer 中的**位置编码**机制。

---

## 1. 理论解析

### 1.1 核心问题：为什么需要位置编码？

**RNN/LSTM** 按顺序处理序列，天然具有位置感知能力：
```
x1 -> x2 -> x3 -> x4  (顺序处理)
```

**Transformer** 使用自注意力并行处理所有位置：
```
x1, x2, x3, x4  (同时处理，无顺序信息)
```

**问题**：对于 Transformer，"我爱你" 和 "你爱我" 的注意力计算结果完全相同！

**解决方案**：在输入 Embedding 中注入位置信息。

### 1.2 数学原理：正弦/余弦位置编码

Transformer 原论文使用固定的正弦/余弦函数生成位置编码：

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

其中：
- $pos$：位置索引 (0, 1, 2, ...)
- $i$：维度索引 (0, 1, 2, ..., d_model/2 - 1)
- $d_{model}$：模型维度

**关键洞察**：
- 偶数维度使用 $\sin$，奇数维度使用 $\cos$
- 不同维度对应不同频率的波

### 1.3 波长的几何意义

分母 $10000^{2i/d_{model}}$ 控制波长：

| 维度 $i$ | 波长 | 特点 |
|:--------:|:-----|:-----|
| $i=0$ | $2\pi$ | 高频，变化快，区分相邻位置 |
| $i=d/2$ | $2\pi \times 10000$ | 低频，变化慢，区分远距离位置 |

**类比二进制编码**：
```
位置 0: 0 0 0 0
位置 1: 0 0 0 1  <- 最低位变化最快
位置 2: 0 0 1 0
位置 3: 0 0 1 1
位置 4: 0 1 0 0  <- 高位变化慢
```

正弦编码是二进制的**连续版本**，每个位置都有唯一的"指纹"。

**优势**：
1. 可以外推到训练时未见过的更长序列
2. $PE_{pos+k}$ 可以表示为 $PE_{pos}$ 的线性函数（相对位置可学习）

---

## 2. 代码实现

In [ ]:
import math

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
class PositionalEncoding(nn.Module):
    """正弦/余弦位置编码。

    使用固定的正弦和余弦函数为序列中的每个位置生成唯一编码。

    Attributes:
        dropout: Dropout 层。
        pe: 位置编码矩阵，形状 (1, max_len, d_model)。

    Example:
        >>> pe = PositionalEncoding(d_model=512, max_len=100)
        >>> x = torch.randn(2, 50, 512)
        >>> output = pe(x)  # 形状不变
    """

    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1) -> None:
        """初始化位置编码模块。

        Args:
            d_model: 模型维度。
            max_len: 支持的最大序列长度。
            dropout: Dropout 概率。
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 创建位置编码矩阵
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Log Trick: 使用 exp(-log) 代替直接除法，提高数值稳定性
        # 原式: 1 / 10000^(2i/d_model) = exp(-2i * log(10000) / d_model)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model)
        )

        # 偶数维度: sin, 奇数维度: cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # 增加 batch 维度: (max_len, d_model) -> (1, max_len, d_model)
        pe = pe.unsqueeze(0)

        # register_buffer: 不是参数，但会随模型保存/加载
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """将位置编码加到输入上。

        Args:
            x: 输入张量，形状 (batch, seq_len, d_model)。

        Returns:
            添加位置编码后的张量，形状不变。
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)

---

## 3. 可视化验证

In [ ]:
# 生成位置编码矩阵
max_len = 100
d_model = 128

pe_module = PositionalEncoding(d_model=d_model, max_len=max_len, dropout=0.0)
pe_matrix = pe_module.pe.squeeze(0).numpy()  # (max_len, d_model)

print(f"位置编码矩阵形状: {pe_matrix.shape}")

In [ ]:
# 绘制热力图
plt.figure(figsize=(12, 8))
plt.imshow(pe_matrix, cmap="RdBu", aspect="auto", vmin=-1, vmax=1)
plt.colorbar(label="Encoding Value")
plt.xlabel("Dimension")
plt.ylabel("Position")
plt.title("Positional Encoding Heatmap (max_len=100, d_model=128)")
plt.tight_layout()
plt.show()

### 热力图解读

观察上图中的**波浪状纹理**：

1. **左侧（低维度）**：波长短，变化快，呈现密集的条纹
2. **右侧（高维度）**：波长长，变化慢，呈现稀疏的条纹
3. **每一行**：代表一个位置的唯一"指纹"
4. **相邻行**：编码值相似但不完全相同

这种设计让模型能够：
- 通过低维度区分相邻位置
- 通过高维度感知全局位置

In [ ]:
# 绘制不同维度的正弦波
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
dims = [0, 10, 50, 100]

for ax, dim in zip(axes.flat, dims):
    ax.plot(pe_matrix[:, dim], label=f"dim={dim}")
    ax.set_xlabel("Position")
    ax.set_ylabel("Value")
    ax.set_title(f"Dimension {dim}")
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-1.2, 1.2)

plt.suptitle("不同维度的位置编码波形", fontsize=14)
plt.tight_layout()
plt.show()

### 波形分析

- **Dimension 0**：周期约为 $2\pi \approx 6$，变化最快
- **Dimension 100**：周期远大于 100，几乎是单调变化
- 不同维度的组合构成每个位置的唯一编码

---

## 4. 总结

| 要点 | 说明 |
|:-----|:-----|
| **目的** | 为并行计算的 Transformer 注入位置信息 |
| **方法** | 正弦/余弦函数，不同维度不同频率 |
| **register_buffer** | 存储非参数张量，随模型保存 |
| **Log Trick** | 提高大指数计算的数值稳定性 |

**下一步**：在 `encoder.ipynb` 中，我们将把位置编码与多头注意力组合成完整的 Encoder 层。